In [72]:
import time
import math
import random
from collections import deque

import numpy as np
import pickle
import gym

from keras import layers, models

## Classic qlearn

In [ ]:
def get_state(obs_vec):
    """
    Encodes an observed state to a number between 0 and 99999 for a given observation.
    :param obs_vec: Observation vector given from the environment
    :return: integer between 0 and 9999 describing the current state
    """
    disc_vec = (obs_vec / np.array([2.4, 2, 12*math.pi/180, 2]) + 1)/2
    disc_vec = np.round(disc_vec, 1)
    return int(np.array([1000, 100, 10, 1]).dot(disc_vec))

In [ ]:
class CartPole:
    """
    RL classic Q-learning implementation of cart pole problem, balancing a pole upright on a cart on a friction-less
    surface. The "Gym" package from OpenAI is used as the gym environment for the RL method. This simple Q-learning
    method is quite inefficient and under-performing, but illustrates the essential idea of the RL approach.
    """

    def __init__(self):
        """
        Initiate the CartPole object and the Q-matrix with 0s
        """
        self.Q = np.zeros((10000, 2))

    def train(self, n, save_file=None, monitor=False, printout=False):
        """
        Training the agent on the cart pole environment. The environment is reset at time 200, if the angle of the pole
        exceeds +-12 degrees or cart position outside +-2.4.
        :param n: Number of training episodes
        :param save_file: If the Q-table should be saved or not to .pkl file
        :param monitor: If the environment should be rendered or not
        :return: None
        """
        env = gym.make('CartPole-v0')

        # Initial hyper parameter values
        eps = .5
        alpha = 1
        gamma = .99

        # Save the times to a 1000 dim vector for calculating the average to monitor the training
        avg = np.zeros(1000)
        for i_episode in range(1, n):

            # Alpha and epsilon decreasing with the number of episodes
            if alpha > .2:
                alpha *= .9995
            if eps > .1:
                eps *= .9996

            # Get the initial state of the cart pole
            observation = env.reset()
            state = get_state(np.array(observation))

            for t in range(1, 1000):

                # Monitor every 10th episode if monitor param is True
                if monitor and (i_episode % 100 == 0):
                    env.render()
                    time.sleep(.02)

                # Random variable to decide if the agent should explore or exploit
                explore = random.random()
                if explore < eps:
                    action = env.action_space.sample()
                else:
                    action = np.argmax(self.Q[state])

                    # If the maximum is 0 i.e. no value has been set yet to the row, choose action at random
                    if not self.Q[state, action]:
                        action = random.randint(0, 1)

                # Take the decided action and time step
                observation, reward, done, info = env.step(action)
                new_state = get_state(np.array(observation))

                # Tweaking of the reward given: -1 if the pole falls and larger reward when pole angle close to 0
                reward -= 5*abs(observation[2])

                # Update the Q-value
                self.Q[state, action] = self.Q[state, action] + alpha*(reward + gamma*np.amax(self.Q[new_state]) -
                                                                       self.Q[state, action])
                if done:
                    avg[i_episode % 1000] = t
                    if printout and (i_episode % 1000 == 0):
                        print("Iteration: {}".format(i_episode))
                        print("Episode finished at average time t = {}".format(np.mean(avg)))
                    break

                state = new_state

        # Save the Q-table
        if save_file:
            q_file = open(save_file, "wb")
            pickle.dump(self.Q, q_file)
            q_file.close()

        env.close()

    def demo(self, n):
        """
        Demonstrates the cart pole performance using the loaded Q-matrix
        :param n: Number of episodes to demonstrate
        :return: None
        """
        env = gym.make('CartPole-v0')

        for i in range(n):
            observation = env.reset()
            state = get_state(np.array(observation))

            for t in range(1, 1000):
                env.render()
                time.sleep(.02)

                # Choose action from Q-table
                action = np.argmax(self.Q[state])
                observation, reward, done, info = env.step(action)
                state = get_state(np.array(observation))

                if done:
                    print("Episode finished after {} time steps".format(t + 1))
                    break
        env.close()

    def load_q_file(self, file_name):
        """
        Load a Q-matrix using a saved .pkl file
        :param file_name: Path to the saved Q-matrix file
        :return: None
        """
        f = open(file_name, "rb")
        self.Q = pickle.load(f)
        f.close()

In [ ]:
cp = CartPole()
# cp.train(10000, save_file="Q_tables/Q1e4states.pkl", monitor=False)
cp.load_q_file("cartpole/Q_tables/Q1e4states.pkl")
cp.demo(5)

## Deep qlearning

In [164]:
class DeepCartPole:
    

    def __init__(self):
        self.Q = None
        self.Q_target = None

    def train(self, env, eps=.5, alpha=1, gamma=.99, monitor=False):
        MINI_SIZE = 64

        model = models.Sequential()
        model.add(layers.Dense(10, activation='relu',
                               input_shape=env.observation_space.shape))
        model.add(layers.Dense(10, activation='relu'))
        model.add(layers.Dense(env.action_space.n, activation='linear'))
        model.compile(loss='mse', optimizer='rmsprop')

        Q = model
        Q_target = model

        samples = deque(maxlen=1000)

        for i_episode in range(1, 100):

            # Get the initial state of the cart pole
            observation = env.reset()
            state = np.array(observation)

            for t in range(1, 1000):

                # Monitor every 10th episode if monitor param is True
                if monitor and (i_episode % 10 == 0):
                    env.render()
                    time.sleep(.02)

                # Random variable to decide if the agent should explore or exploit
                explore = random.random()
                if explore < eps:
                    action = env.action_space.sample()
                else:
                    action = np.argmax(Q.predict(np.array([observation])))

                # Take the decided action and time step
                observation, reward, done, info = env.step(action)
                new_state = np.array(observation)
                sars = SARS(state, action, reward, new_state)
                samples.append(sars)

                if done:
                    break
                    #avg[i_episode % 1000] = t
#                     if printout and (i_episode % 1000 == 0):
#                         print("Iteration: {}".format(i_episode))
#                         print("Episode finished at average time t = {}".format(
#                             np.mean(avg)))
#                     break
                state = new_state
            
            if len(samples) > MINI_SIZE:
                mini_batch = random.sample(samples, MINI_SIZE)
                states = []
                rewards = []
                actions = []
                new_states = []

                for sars in mini_batch:
                    states.append(sars.s)
                    rewards.append(sars.a)
                    actions.append(sars.r)
                    new_states.append(sars.sn)
                
                states = np.array(states)
                rewards = np.array(rewards)
                actions = np.array(actions)
                new_states = np.array(new_states)
                
                Q_pred = Q.predict(states)
                labels = Q_pred
                label_idxs = np.argmax(Q_target.predict(new_states), axis=1)
                label_values = rewards + gamma * np.amax(Q_target.predict(new_states), axis=1)
                
                for i, (idx, value) in enumerate(zip(label_idxs, label_values)):
                    labels[i][idx] = value
                
                Q.fit(states, labels, epochs=1000, verbose=0, batch_size = 64)
                    



class SARS:
    def __init__(self, s, a, r, sn):
        self.s = s
        self.a = a
        self.r = r
        self.sn = sn

In [165]:
env = gym.make('CartPole-v0')
deep = DeepCartPole()
deep.train(env)

KeyboardInterrupt: 